In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import nltk
import sys
import datetime
sns.set(font_scale=2)
# pd.set_option('display.max_columns', 500)


In [ ]:
df2 = pd.read_csv("../input/gufhtugu-publications-dataset-challenge/GP Orders - 4.csv",encoding="utf-8", delimiter=',')


In [ ]:
print( df2.shape)
df2['Book Name'].dtype

In [ ]:
df2

# Status Frequency 

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(10, 10))
# splot=sns.barplot(x=df2['Order Status'].unique(),y=df2['Order Status'].value_counts(),orient="v")
# for i, p in enumerate(ax.patches):
#     ax.annotate(format(p.get_height(), '.1f'), 
#                    (p.get_x() + p.get_width() / 2., p.get_height()), 
#                    ha = 'center', va = 'center', 
#                    xytext = (0, 9), 
#                    textcoords = 'offset points')
# =============Below is same==================================
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
df2['Order Status'].value_counts().plot.bar()
for i, p in enumerate(ax.patches):
    ax.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')


# Data Summary

In [ ]:
# @navid
statistics_of_data = []
for col in df2.columns:
    print(col)
    statistics_of_data.append((col,
                             df2[col].nunique(),
                             df2[col].isnull().sum()*100/df2.shape[0],
                             df2[col].value_counts(normalize=True, dropna=False).values[0] * 100, 
                             df2[col].dtype
                            
                             ))
stats_df = pd.DataFrame(statistics_of_data, columns=['Feature', 'Uniq_val', 'missing_val', 'val_biggest_cat', 'type'])
stats_df

In [ ]:

# df2.drop(df2[df2['Book Name'].isnull()].index,axis=0,inplace=True)
# df2.shape
# df2['Book Name']=df2['Book Name'].to_string()
df2['Book Name']=df2['Book Name'].apply(lambda x: str(x))
df2['City (Billing)']=df2['City (Billing)'].apply(lambda x:str(x))
df2['Book Name'] = df2['Book Name'].str.lower()
df2['City (Billing)'] = df2['City (Billing)'].str.lower()

# Separating Books 

In [ ]:
df2.columns=['order_name', 'order_status', 'book_name', 'order_date','city_billing']
df2.tail(20)
from itertools import chain

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split('/')))

# calculate lengths of splits
lens = df2['book_name'].str.split('/').map(len)

# create new dataframe, repeating or chaining as appropriate
df2 = pd.DataFrame({'order_name': np.repeat(df2['order_name'], lens),
                    'order_status': np.repeat(df2['order_status'], lens),
                    'book_name': chainer(df2['book_name']),
                    'order_date': np.repeat(df2['order_date'], lens),
                    'city_billing': np.repeat(df2['city_billing'], lens)})
df2.reset_index(inplace=True,drop=True)
df2

#  Prosessiong-1 (Most Fequenty cities)

In [ ]:
%%time

pakistan_top_cities = ['karachi', 'lahore', 'sialkot', 'faisalabad', 'rawalpindi',
       'peshawar', 'saidu sharif', 'multan', 'gujranwala', 'islamabad',
       'quetta', 'bahawalpur', 'sargodha', 'new mirpur', 'chiniot',
       'sukkur', 'larkana', 'shekhupura', 'jhang', 'rahimyar khan',
       'gujrat', 'kasur', 'mardan', 'mingaora', 'dera ghazi khan',"dgk"
       'nawabshah', 'sahiwal', 'mirpur khas', 'okara', 'burewala',
       'jacobabad', 'saddiqabad', 'kohat', 'muridke', 'muzaffargarh',
       'khanpur', 'gojra', 'bahauddin', 'abbottabad', 'dadu',
       'khuzdar', 'pakpattan', 'tando allahyar', 'vihari', 'jaranwala',
       'kamalia', 'kot addu', 'nowshera', 'swabi', 'dera ismail khan',
       'chaman', 'charsadda', 'kandhkot', 'hasilpur', 'muzaffarabad',
       'mianwali', 'jalalpur","jattan', 'bhakkar', 'zhob', 'kharian',
       'mian channun', 'jamshoro', 'pattoki', 'harunabad',
       'toba tek singh', 'shakargarh', 'hujra", "shah", "muqim', 'kabirwala',
       'mansehra', 'lala musa', 'nankana sahib', 'bannu', 'timargara',
       'parachinar', 'gwadar', 'abdul hakim', 'hassan", "abdal', 'tank',
       'hangu', 'risalpur cantonment', 'karak', 'kundian', 'umarkot',
       'chitral', 'dainyor', 'kulachi', 'kotli', 'gilgit',
       'hyderabad', 'narowal', 'khairpur', "mir’s", 'khanewal', 'jhelum',
       'haripur', 'shikarpur', 'rawala kot', 'hafizabad', 'lodhran',
       'malakand', 'attock', 'batgram', 'matiari', 'ghotki',
       'firoz','naushahro', 'alpurai', 'bagh', 'daggar', 'bahawalnagar',
       'leiah', 'tando muhammad khan', 'chakwal', 'khushab', 'badin',
       'lakki', 'rajanpur', 'dera allahyar', 'shahdad kot', 'pishin',
       'sanghar', 'upper dir', 'thatta', 'dera murad jamali', 'kohlu',
       'mastung', 'dasu', 'athmuqam', 'loralai', 'barkhan',
       'musa khel bazar', 'ziarat', 'gandava', 'sibi', 'dera bugti',
       'eidgah', 'turbat', 'uthal', 'chilas', 'kalat', 'panjgur', 'gakuch',
       'qila', 'saifullah', 'kharan', 'aliabad', 'awaran', 'dalbandin']
def city(x):
    if x in pakistan_top_cities :
        return True
    else:
        return False
new=df2.city_billing.apply(city)
found=new[new==True]
not_found=new[new==False] #These cities were not found in paK_cities. It is seprated out and wil be dealth with later.

len(found)
df2.loc[found.index.to_list(),'city_billing'].nunique()

In [ ]:
not_found=df2.loc[not_found.index.to_list(),'city_billing']
not_found.size

In [ ]:
df2.nunique()

In [ ]:
df2['order_date']= pd.to_datetime(df2['order_date'])
df2['year'] = df2['order_date'].apply(lambda x : x.year)
df2['month'] = df2['order_date'].apply(lambda x : x.month)
df2['day'] = df2['order_date'].apply(lambda x : x.day_name())
df2['weekday'] = df2['order_date'].apply(lambda x : x.weekday())


In [ ]:
# def cat_plot(col,status):
#     fig, ax = plt.subplots(1, 1, figsize=(15, 10))
#     print(col)
#     df=df2.loc[df2['order_status']==status,['order_status',col]].groupby(col,as_index=False).count()
#     if df.shape[0] >20:
#         df=df.nlargest(n=10,columns='order_status')
#     sns.barplot(y=col,x='order_status',data=df,orient='h' )
#     for p in ax.patches:
# #         value_count = '{:.1f}'.format(p.get_width())
#         value_count = p.get_width()
#         x = p.get_x() + p.get_width() + 0.02
#         y = p.get_y() + p.get_height()/2
#         ax.annotate(value_count, (x, y))
#     plt.ylabel(f'{col}')
#     plt.xlabel(f'Number of {status} Orders')
#     plt.show()
    

In [ ]:
# nf['city_billing'] = nf['city_billing'].str.replace('\d+', '')
# nf['city_billing'] = nf['city_billing'].str.replace('pakistan', '')
# nf['city_billing'] = nf['city_billing'].str.replace('city', '')
# nf['city_billing'] = nf['city_billing'].str.replace('?', '')

# nf['city_billing'].tail(20)
# nf['city_billing'].to_string()


In [ ]:
not_found[0:31]

In [ ]:
%%time
def city(x):
    if '?' in x or len(x)==3 :
        return 'others'
    else:
        return x
not_found=not_found.apply(city)
print(len(not_found))
df2.loc[not_found.index.to_list(),'city_billing']=not_found
df2.nunique()

In [ ]:
not_found[0:31]

In [ ]:
not_found=not_found[not_found!='others']
not_found.size

In [ ]:
%%time
single_word_cities = df2[df2['city_billing'].str.split().apply(len) == 2]["city_billing"].unique()
single_word_cities[:20]
def clean_city(row):
    address = row.city_billing.split()
    add = set()
    for a in address:
        a = a.strip()
        if a:
            add.add(a)
    for city in pakistan_top_cities:
        if row.city_billing.__contains__(city):
            return city
        
    for a in add:
        for c in pakistan_top_cities:
            if nltk.edit_distance(a, c) <= 5: # considering spelling mistakes upto 5 letters
                return c
    return row.city_billing
nf2=pd.DataFrame(not_found)
nf2["city_billing"]=nf2.apply(clean_city, axis=1)
nf2.nunique()


In [ ]:
df2.loc[nf2.index.to_list(),'city_billing']=nf2
df2.nunique()


In [ ]:
def cat_plot(col,status):
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    print(col)
    df=df2.loc[df2['order_status']==status,['order_status',col]].groupby(col,as_index=False).count()
    if df.shape[0] >20:
        df=df.nlargest(n=10,columns='order_status')
    sns.barplot(y=col,x='order_status',data=df,orient='h' )
    for p in ax.patches:
#         value_count = '{:.1f}'.format(p.get_width())
        value_count = p.get_width()
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(value_count, (x, y))
    plt.ylabel(f'{col}')
    plt.xlabel(f'Number of {status} Orders')
    plt.show()
    

# EDA
# TOP BOOKS SOLD

In [ ]:
cat_plot('book_name','Completed')
# pass Returned or Cancles for their status

# TOP CITIES FOR BOOK SALE

In [ ]:
cat_plot('city_billing','Completed')

# TOP CITIES OF RETURNED BOOK

In [ ]:
cat_plot('city_billing','Returned')

# TOP YEAR FOR BOOK SALE

In [ ]:
cat_plot('year','Completed')

# TOP MONTH FOR BOOK SALE

In [ ]:
cat_plot('month','Completed')

# Least Books Sold

In [ ]:
cat_plot('month','')

In [ ]:
# ran=np.random.randint(0,nf2.shape[0],10)
# for i in range(9):
#     ran=np.random.randint(0,nf2.shape[0],10)
#     cur=pd.DataFrame(nf2.loc[ran,'city_billing'])
#     cur=cur.reset_index(drop=True)
#     if i!=0:
#         cur=pd.concat([old,cur],axis=1)
#     old=cur
# cur


In [ ]:
df2.city_billing.unique()
